In [1]:
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
import numpy as np
import random

In [2]:
# Produto escalar de duas listas numéricas
dotp = lambda v1, v2: sum([x*y for x,y in zip(v1, v2)])

In [3]:
# Iniciar população aleatóriamente
def startPop(size):
    pop = []
    for i in range(size):
        # Nosso alfabeto será inteiros entre 4 e 188
        pop.append(list(np.random.random_integers(4,177,5)))

    # Adicionar coluna de fitness
    for i in pop:
        i.append(fitness(i))

    # Ordenar por fitness
    pop.sort(key=lambda x: x[5], reverse=True)
    return pop

In [4]:
# Cálculo dafitness dado um indivíduo
def fitness(vec):
    pesos = [2, 4, 5, 8, 12] #pesos de cada tipo
    valores = [3, 6, 10, 18, 25] #valor de cada tipo
    pesoMax = 500

    # O peso e o valor de cada indivíduo
    # será o produto escalar entre o
    # vetor de pesos e valores e a quantidade 
    # de cada tipo no indivíduo
    pesoInd = dotp(vec, pesos)
    valorInd = dotp(vec, valores)

    if (pesoInd <= pesoMax):
        return valorInd
    # Punição para o caso de o peso exceder o max
    else:
        return valorInd - 24 * (pesoInd - pesoMax)

In [5]:
# Seleção de pais por torneio
def torneio(pop, tx, nCandidatos):
    popPais = []
    # Sempre seleciona um n par de pais
    nFilhos = 2 * int(len(pop) * tx / 2)

    for i in range(nFilhos):
        # Amostrar candidatos aleatóriamente
        candidatos = random.sample(pop, nCandidatos)
        #print(pop)
        popPais.append(candidatos[0])
        bestFitness = candidatos[0][5]
        #selecionar apenas o de maior fitness
        for j in candidatos:
            if(j[5] > bestFitness):
                popPais = popPais[:-1]
                popPais.append(j)
                bestFitness = j[5]
                
    return popPais

In [26]:
# Cruzamento por máscara binária dada a população de pais
def cruzamento(popPais):
    popFilhos = []
    # Cruzar população de pais dois a dois
    for i,k in zip(popPais[0::2], popPais[1::2]):
        filho1 = []
        filho2 = []
        # Máscara binária aleatória
        mask = np.random.randint(2, size=5)
        # Cruzamento de acordo com a máscara
        for j in range(len(mask)):
            if(mask[j]):
                filho1.append(i[j])
                filho2.append(k[j])
            else:
                filho1.append(k[j])
                filho2.append(i[j])
        # Cálcular a fitness dos filhos
        filho1.append(fitness(filho1))
        filho2.append(fitness(filho2))
        popFilhos.append(filho1)
        popFilhos.append(filho2)
    return popFilhos

In [7]:
#Adicionar filhos na população
def inserirFilhos(pop, popFilhos):
    # Eliminar piores indivíduos da população
    pop = pop[:-len(popFilhos)]
    #inserir indivíduos filhos
    for i in popFilhos:
        pop.append(i)
    #organizar a população por fitness
    pop.sort(key=lambda x: x[5], reverse=True)
    return pop

In [8]:
# Mutar n indivíduos aleatóriamente
def mutacao(pop, tx):
    nMutados = int(len(pop) * tx)
    elitismo = True

    for i in range(nMutados):
        # Se houver elitismo, nunca se muta o melhor indivíduo
        #index do indivíduo a ser mutado
        if (elitismo):
            index1 = np.random.random_integers(1, len(pop) - 1, 1)[0]
        else:
            index1 = np.random.random_integers(0, len(pop) - 1, 1)[0]
        #index do gene a ser mutado
        index2 = np.random.random_integers(0, 4, 1)[0]
        pop[index1][index2] = np.random.random_integers(4,188,1)[0]
        # Recálculo da fitness
        pop[index1][5] = fitness(pop[index1])
    # Reornganização da população
    pop.sort(key=lambda x: x[5], reverse=True)
    return pop

In [9]:
def Roleta(pop, tx, nt):
    popPais = []
    
    n = 2 * (len(pop) * tx / 2)
    
    for i in range(int(n)):
        rol = 0
        better = [] # guarda o fitness
        miniPop = [] # guarda a populacao selecionada
        val = [] # guarda as porcentagens
        for j in range(int(nt)):
            miniPop.append(pop[random.randint(0, len(pop) - 1)])
            
        for j in miniPop:
            better.append(j[0])
            
        for j in range(int(nt)):
            percent = (sum(better)/better[j]) * 100
            val.append(percent)
            
        roleta = random.randint(0,99)
        
        for j in range(int(nt)):
            rol += val[j]
            
            if roleta <= rol:
                popPais.append(miniPop[j])

    return popPais

In [29]:
# Listas que serão plotadas
bestIndFit = []
mediumFit = []

pop = startPop(100)

nGerações = 100
taxaDeCruzamento = 0.5
taxaDeMutacao = 0.2

for i in range(nGerações):
    popPais = torneio(pop, taxaDeCruzamento, 3)
    popFilhos = cruzamento(popPais)
    pop = inserirFilhos(pop, popFilhos)
    pop = mutacao(pop, taxaDeMutacao)
    print(f'Geração {i} -- Melhor Indivíduo: {pop[0]} -- Peso: {dotp([2, 4, 5 , 8, 12], pop[0])}\n')
    bestIndFit.append(pop[0][5])
    mediumFit.append(np.mean(pop, axis=0)[5])

# Plot da fitness
fig = Figure()
FigureCanvas(fig)
ax = fig.add_subplot(111)
ax.plot(bestIndFit, label='Fitness do Melhor Indivíduo')
ax.plot(mediumFit, '-.', label='Fitness Média da População')
#ax.set_title('Fitness ao Longo das Gerações')
ax.set_xlabel('Geração')
ax.set_ylabel('Fitness')
ax.set_yscale('symlog')
ax.set_xlim(0,99)
ax.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc='center',
       ncol=2, borderaxespad=0)
#fig.savefig('fit_Roleta_mascara.png')

C:\Users\lidiaxp\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: This function is deprecated. Please call randint(4, 177 + 1) instead
  
C:\Users\lidiaxp\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: This function is deprecated. Please call randint(1, 99 + 1) instead
  # Remove the CWD from sys.path while we load stuff.
C:\Users\lidiaxp\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: This function is deprecated. Please call randint(0, 4 + 1) instead
  
C:\Users\lidiaxp\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: This function is deprecated. Please call randint(4, 188 + 1) instead
  from ipykernel import kernelapp as app


Geração 0 -- Melhor Indivíduo: [5, 13, 51, 48, 30, -11247] -- Peso: 1061

Geração 1 -- Melhor Indivíduo: [19, 49, 34, 36, 16, -7477] -- Peso: 884

Geração 2 -- Melhor Indivíduo: [126, 18, 40, 17, 14, -6330] -- Peso: 828

Geração 3 -- Melhor Indivíduo: [66, 6, 38, 32, 5, -2573] -- Peso: 662

Geração 4 -- Melhor Indivíduo: [19, 6, 38, 32, 5, -458] -- Peso: 568

Geração 5 -- Melhor Indivíduo: [19, 6, 38, 32, 5, -458] -- Peso: 568

Geração 6 -- Melhor Indivíduo: [19, 16, 40, 17, 5, 984] -- Peso: 498

Geração 7 -- Melhor Indivíduo: [5, 13, 7, 36, 7, 986] -- Peso: 469

Geração 8 -- Melhor Indivíduo: [5, 13, 7, 36, 7, 986] -- Peso: 469

Geração 9 -- Melhor Indivíduo: [19, 20, 7, 32, 7, 998] -- Peso: 493

Geração 10 -- Melhor Indivíduo: [19, 20, 7, 32, 7, 998] -- Peso: 493

Geração 11 -- Melhor Indivíduo: [5, 20, 26, 17, 12, 1001] -- Peso: 500

Geração 12 -- Melhor Indivíduo: [5, 20, 26, 17, 12, 1001] -- Peso: 500

Geração 13 -- Melhor Indivíduo: [19, 23, 6, 32, 7, 1006] -- Peso: 500

Geração 